In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import time
from sklearn.metrics import accuracy_score

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['weatherAUS.csv']


Import Dataset

In [2]:
data=pd.read_csv("../input/weatherAUS.csv")

Have a look at the sample data

In [3]:
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


# We need to remove RISK_MM because we want to predict 'RainTomorrow' and RISK_MM can leak some info to our model



In [4]:
data = data.drop(columns=['RISK_MM'],axis=1)

# Taking care of missing data

In [11]:
from sklearn.preprocessing import Imputer
imputer=Imputer(missing_values='NaN',strategy='mean',axis=0)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


#Impute missing values for numerical varibales

In [13]:
df_num_col = ["MinTemp","MaxTemp","Rainfall","Evaporation","Sunshine","WindGustSpeed","WindSpeed9am","WindSpeed3pm","Humidity9am","Humidity3pm","Pressure9am","Pressure3pm","Cloud9am","Cloud3pm","Temp9am","Temp3pm"]
data_num=data[df_num_col]
imputer=imputer.fit(data_num)
data[df_num_col]=imputer.transform(data_num)

#Use encoder for categorical variables

In [14]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder = LabelEncoder()
df_cat_col = ["WindGustDir","WindDir9am","WindDir3pm","RainToday","RainTomorrow","Date","Location"]

#Update NaN values by NA

In [15]:
data_cat=data[df_cat_col].fillna('NA')

#Loop through each categorical variable and update values using LabelEncoder

In [16]:
for i in range(len(data_cat.columns)):
  data_cat.iloc[:,i] = labelencoder.fit_transform(data_cat.iloc[:,i])
  
data[df_cat_col]=data_cat

Get dependent and independent variable from dataset

In [17]:
x=data.iloc[:,0:22].values
y=data.iloc[:,22].values

#Split data set into train and test set

In [18]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

# Feature Scaling

In [19]:
from sklearn.preprocessing import StandardScaler
sc_x=StandardScaler()
x_train=sc_x.fit_transform(x_train)
x_test=sc_x.fit_transform(x_test)

Apply different classification models on the data

#Logistic Regression

In [20]:
from sklearn.linear_model import LogisticRegression
import time
from sklearn.metrics import accuracy_score
t0=time.time()
logreg=LogisticRegression(random_state=0)
logreg.fit(x_train,y_train)
y_pred=logreg.predict(x_test)
score = accuracy_score(y_test,y_pred)
print('Logistic Regression Accuracy :',score)
print('Logistic Regression Time taken :' , time.time()-t0)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy : 0.8446499525299764
Logistic Regression Time taken : 1.509413719177246


#Decision Tree



In [21]:
from sklearn.tree import DecisionTreeClassifier
t0=time.time()
destree=DecisionTreeClassifier(random_state=0)
destree.fit(x_train,y_train)
y_pred=destree.predict(x_test)
score = accuracy_score(y_test,y_pred)
print('Decision Tree Accuracy :',score)
print('Decision Tree Time taken :' , time.time()-t0)

Decision Tree Accuracy : 0.7891627694363375
Decision Tree Time taken : 2.8927671909332275


#Random Forest Classifier


In [22]:
from sklearn.ensemble import RandomForestClassifier
t0=time.time()
rantree=RandomForestClassifier(random_state=0)
rantree.fit(x_train,y_train)
y_pred=rantree.predict(x_test)
score = accuracy_score(y_test,y_pred)
print('Random Tree Accuracy :',score)
print('Random Tree Time taken :' , time.time()-t0)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Random Tree Accuracy : 0.8474981539435282
Random Tree Time taken : 4.046960830688477


Analysis shows both Logical regressor and Random forest regressor have accuracy of 84%.
But performance wise Logical regressor is better since it took only 1.5 Seconds.